<a href="https://colab.research.google.com/github/siliconemonster/IA-2020.1/blob/main/Algoritmo_Genetico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import random

In [9]:
def geraTabuleiro(n):
  tabuleiro = []

  for x in range(n):
    posicaoRainha = random.randint(1, n)
    tabuleiro.append(posicaoRainha)

  return tabuleiro

geraTabuleiro(4)

[4, 3, 4, 1]

In [11]:
def avaliaAtaques(tabuleiro):
  n = len(tabuleiro)
  ataques = 0

  for i in range(n-1): # caso esteja na mesma coluna
    for j in range(i+1, n):
      if (tabuleiro[i] == tabuleiro[j]):
        ataques = ataques + 1

  for i in range(n-1): # caso esteja na mesma diagonal. Para baixo e para cima, respectivamente
    for j in range(i+1, n):
      if ((tabuleiro[i] == tabuleiro[j] + (j - i)) or (tabuleiro[i] == tabuleiro[j] - (j - i))):
        ataques = ataques + 1

  #print("Número de ataques:")
  #print(ataques)
  return ataques

tabuleiro = geraTabuleiro(4)
avaliaAtaques(tabuleiro)

2

In [12]:
def geraPopulacao(numIndividuos, n):
  populacao = []

  for i in range(numIndividuos):
    populacao.append(geraTabuleiro(n)) ##### FALTA LIDAR COM ELEMENTOS REPETIDOS

  return populacao

geraPopulacao(10, 4)

[[2, 4, 1, 1],
 [1, 3, 1, 1],
 [1, 2, 1, 1],
 [3, 3, 1, 4],
 [1, 2, 4, 1],
 [2, 4, 1, 1],
 [4, 1, 2, 3],
 [4, 1, 2, 2],
 [1, 4, 4, 4],
 [1, 1, 1, 4]]

In [13]:
def crossover(pai1, pai2):
  filhos = []
  n = len(pai1)
  pontoCorte = random.randint(0, n-2) #ponto corte
  print(pontoCorte)

  inicio1 = pai1[:pontoCorte+1]
  final1 = pai2[pontoCorte+1:]
  filho1 = inicio1 + final1
  filhos.append(filho1)

  inicio2 = pai2[:pontoCorte+1] 
  final2 = pai1[pontoCorte+1:]
  filho2 = inicio2 + final2
  filhos.append(filho2)


  return filhos

crossover([1, 2, 3, 3], [2, 3, 3, 4])

0


[[1, 3, 3, 4], [2, 2, 3, 3]]

In [15]:
#Troca o aleatorio escolhido por quem tá do lado direito, se o aleatorio for o ultimo troca com o primeiro
def mutacao(individuo):  #### EXISTEM OUTROS TIPOS DE MUTAÇÃO
  n = len(individuo)
  aleatorio = random.randint(0, n-1)
  #print(aleatorio)

  if (aleatorio == n-1):
    aux = individuo[aleatorio]
    individuo[aleatorio] = individuo[0]
    individuo[0] = aux

  else:
    aux = individuo[aleatorio]
    individuo[aleatorio] = individuo[aleatorio + 1]
    individuo[aleatorio + 1] = aux

  return individuo

mutacao([1,2,3,4,5,6,7,8])

[1, 2, 3, 4, 5, 7, 6, 8]

O algoritmo genético:

In [8]:
def genetico(populacao, probCrossover, probMutacao):

  novaPopulacao = []

  while True:
    for i in range(len(populacao)):
      aux1 = random.randint(0, (len(populacao)))
      aux2 = random.randint(0, (len(populacao)))
      pai1 = populacao[aux1]
      pai2 = populacao[aux2] ##### FALTA GARANTIR QUE PAIS NÃO SÃO REPETIDOS
      
      probCalculada = random.uniform(0, probCrossover)
      if probCalculada < probCrossover:
        filho = crossover(pai1, pai2)
      
      probCalculada = random.uniform(0, probMutacao)
      if (probCalculada <= probMutacao):
        filho = mutacao(filho)
      novaPopulacao[i] = filho
    
    populacao = novaPopulacao # acho que isso está errado
    if (condicoesDeParada): # determinar condições de parada
      return melhorElemento # determinar quem é o melhor elemento

populacao = geraPopulacao(10, 4)
genetico(populacao)

TypeError: ignored

In [17]:
def roletaViciada(populacao):
  n = len(populacao)
  roleta = []

  for i in range(n):
    #print(populacao[i])
    r = avaliaAtaques(populacao[i])
    #print(r)
    roleta.append(r)
  
  total = sum(roleta)
  
  #print(roleta, total)
  return roleta, total

roletaViciada(geraPopulacao(10, 4))

([3, 5, 4, 3, 5, 2, 2, 6, 4, 3], 37)

In [18]:
def selecao(populacao):
  #print("Populacao Inicial" + str(populacao))
  n = len(populacao)
  populacao_int = []

  roleta, total = roletaViciada(populacao)

  for i in range(n):
    sorteado = random.randint(0, total)
    #print("Sorteado " + str(sorteado))
    esquerda = 0
    direita = roleta[0]
    j = 0
    while(True):
      if sorteado == 0:
        populacao_int.append(populacao[j])
        break
      elif sorteado > esquerda and sorteado <= direita:
        populacao_int.append(populacao[j])
        break
      esquerda = direita
      direita = direita + roleta[j+1]
      j = j + 1
  
  return populacao_int

selecao(geraPopulacao(6, 4))

[[2, 4, 3, 3],
 [2, 1, 1, 2],
 [2, 3, 1, 4],
 [2, 2, 2, 3],
 [2, 1, 1, 2],
 [2, 1, 1, 2]]